In [1]:
import string
import pandas as pd
import re
import pickle
from tqdm.notebook import tqdm
import unicodedata2
import numpy as np
import random
import os
from sklearn.metrics.pairwise import cosine_similarity

from libs.lda_wrapper import LDA_wrapper
import logging

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
tqdm.pandas()

/home/user/miniconda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
df_socsci = pd.read_csv('/data/datasets/WOS/US/text_clean_socsci.txt')

In [4]:
def save(x, file_name):
    with open(file_name, 'wb') as handle:
        pickle.dump(x, handle, protocol=pickle.HIGHEST_PROTOCOL)

def restore(file_name):
    with open(file_name, 'rb') as handle:
        x = pickle.load(handle)
    return x

In [5]:
#random.sample(range(1234, 99999), 10)

In [6]:
def multiple_runs(df,logger,random_states, dataset= 'socsci',n_batches=100,max_iter = 10,n_components=100):
    LDA = LDA_wrapper(n_batches=n_batches)
    
    for rnd in random_states:
        logger.info('running random state {}'.format(rnd))
        texts = df['text_clean'].values
        lda_model,data_vectorized,vectorizer = LDA.lda(data=texts,max_iter = max_iter,n_components=n_components, random_state = rnd)  
        logger.info('finnish running random state {}'.format(rnd))
        save(lda_model, '../results/lda_runs/lda_model_{}_k{}_seed{}.p'.format(dataset,n_components,rnd))
        logger.info('-------------------------------------------------')
    logger.info('finnish running everything')


In [7]:
logging.basicConfig(filename='ldarobusntess_log', filemode="w",format='%(asctime)s %(message)s', datefmt='%d/%m/%Y %H:%M:%S',level=logging.INFO)
logger = logging.getLogger()

random_states = [79613, 5246, 99257, 34310, 68163, 3161, 45052, 10207, 39373, 80243]
#random_states = [45052, 10207, 39373, 80243]

multiple_runs(df_socsci,logger = logger,random_states = random_states,dataset= 'socsci',n_batches=100,max_iter = 10,n_components=300)

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/user/miniconda/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/user/miniconda/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 558, in run
    self.terminate_broken(bpe)
  File "/home/user/miniconda/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 736, in terminate_broken
    self.kill_workers()
  File "/home/user/miniconda/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 766, in kill_workers
    recursive_terminate(p)
  File "/home/user/miniconda/lib/python3.8/site-packages/joblib/externals/loky/backend/utils.py", line 28, in recursive_terminate
    _recursive_terminate_without_psutil(process)
  File "/home/user/miniconda/lib/python3.8/site-packages/joblib/externals/loky/backend/utils.py", line 53, in _recursive_terminate_without_psutil
    _recursive_terminate(process.pid)
  File "/ho

Traceback (most recent call last):
  File "/home/user/miniconda/lib/python3.8/site-packages/sklearn/decomposition/_lda.py", line 531, in partial_fit
    self._em_step(X[idx_slice, :],
  File "/home/user/miniconda/lib/python3.8/site-packages/sklearn/decomposition/_lda.py", line 465, in _em_step
    _dirichlet_expectation_2d(self.components_))
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/user/miniconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3427, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-a74a34c974c6>", line 7, in <module>
    multiple_runs(df_socsci,logger = logger,random_states = random_states,dataset= 'socsci',n_batches=100,max_iter = 10,n_components=300)
  File "<ipython-input-6-2d4b4bed4e77>", line 7, in multiple_runs
    lda_model,data_vectorized,vectorizer = LDA.lda(data=texts,max_iter = max_iter,n_components=n_co

TypeError: object of type 'NoneType' has no len()

In [9]:
restore('../results/lda_runs/lda_model_socsci_k300_seed10207.p')

LatentDirichletAllocation(learning_method='online', n_components=300, n_jobs=-1,
                          random_state=10207, verbose=1)

In [7]:
models_path = os.listdir('../results/lda_runs/')
model_health_path = '../results/lda_model_health_k200.p'

In [8]:
vectorizer_socsci = restore( '../results/vectorizer_socsci.p')
vectorizer_health = restore( '../results/vectorizer_health.p')


In [9]:
def transform_data(df,model_path,vectorizer):    
    texts = df.text_clean.values
    lda_model = restore(model_path)
    data_vectorized = vectorizer.transform(texts)
    doc_dist = lda_model.transform(data_vectorized)
    l2_norm_docs = np.linalg.norm(doc_dist, axis = 1)
    return l2_norm_docs

In [10]:
# Socsci runs
l2_norms = {}

for path in models_path:
    seed = re.search('seed(.*?).p', path).group(1)
    model_path = '../results/lda_runs/' + path

    l2_norm_docs = transform_data(df_socsci, model_path, vectorizer_socsci)

    l2_norms[seed] = l2_norm_docs

In [11]:
#health model
model_path = '../results/lda_model_health_k200.p'

l2_norm_docs = transform_data(df_socsci, model_path, vectorizer_health)
l2_norms['health'] = l2_norm_docs

In [12]:
#random dirichlet ditribution

random_dist = np.random.dirichlet(np.ones(300),size=l2_norms['34310'].shape[0])
ls_norm_random_dist = np.linalg.norm(random_dist, axis = 1)

l2_norms['random'] = ls_norm_random_dist

In [13]:
l2_norms_df = pd.DataFrame.from_dict(l2_norms, orient='index')

#should be 11xN

In [14]:
l2_norms_df

,0,1,2,3,4,5,6,7,8,9,...,283579,283580,283581,283582,283583,283584,283585,283586,283587,283588
34310,0.528003,0.372632,0.376989,0.519090,0.552922,0.666891,0.501106,0.365903,0.499789,0.442269,...,0.243539,0.234802,0.278083,0.325122,0.269385,0.230820,0.198172,0.209174,0.272705,0.206274
79613,0.534204,0.657670,0.245066,0.444257,0.382167,0.316217,0.763955,0.374204,0.580103,0.307858,...,0.273492,0.203015,0.223286,0.267529,0.202678,0.257276,0.195496,0.198273,0.222563,0.275976
3161,0.801637,0.537860,0.264575,0.848503,0.414444,0.396722,0.378861,0.590479,0.538120,0.469056,...,0.245083,0.206602,0.198026,0.293468,0.276539,0.243448,0.191659,0.213457,0.202953,0.224807
5246,0.607302,0.659719,0.314166,0.718966,0.554533,0.286502,0.465082,0.348229,0.382415,0.643243,...,0.270659,0.199950,0.224526,0.240063,0.250405,0.265927,0.271049,0.190517,0.211497,0.243432
39373,0.530086,0.506708,0.289649,0.724667,0.352222,0.256915,0.382177,0.462292,0.332010,0.504771,...,0.297194,0.210317,0.226572,0.234513,0.206499,0.230798,0.245062,0.209981,0.186404,0.262097
10207,0.336171,0.614528,0.295290,0.480316,0.299741,0.445723,0.544065,0.446864,0.377219,0.315265,...,0.231864,0.226187,0.252000,0.254279,0.193731,0.221757,0.198298,0.192026,0.241258,0.250378
68163,0.669664,0.464109,0.292984,0.529992,0.399905,0.401985,0.725263,0.434833,0.472343,0.412282,...,0.346633,0.206673,0.223133,0.271856,0.256763,0.274313,0.184637,0.221579,0.221203,0.210539
80243,0.549956,0.333410,0.331237,0.431755,0.410316,0.537902,0.629523,0.566604,0.562285,0.513579,...,0.259982,0.168049,0.278319,0.267295,0.260211,0.349950,0.218331,0.214136,0.203207,0.223133
45052,0.500921,0.561760,0.275317,0.487173,0.530171,0.514609,0.546503,0.422351,0.471126,0.325276,...,0.241437,0.215254,0.236741,0.326113,0.267795,0.212976,0.200303,0.189705,0.229156,0.217073
99257,0.366764,0.536983,0.264746,0.406480,0.445542,0.345592,0.457646,0.449023,0.381289,0.589079,...,0.343583,0.188052,0.212507,0.228323,0.195329,0.297253,0.204336,0.206089,0.219774,0.220093


In [52]:
cosine_similarities = cosine_similarity(l2_norms_df)

In [53]:
cosine_similarities_df = pd.DataFrame(cosine_similarities,index = l2_norms_df.index, columns = l2_norms_df.index )

In [55]:
cosine_similarities_df

,34310,79613,3161,5246,39373,10207,68163,80243,45052,99257,health,random
34310,1.000000,0.988142,0.987453,0.987409,0.987182,0.987452,0.988589,0.987683,0.988492,0.988120,0.980510,0.980422
79613,0.988142,1.000000,0.987597,0.986711,0.988790,0.988069,0.988719,0.987677,0.987590,0.989830,0.977851,0.978529
3161,0.987453,0.987597,1.000000,0.988627,0.987392,0.988808,0.988612,0.988219,0.987714,0.988696,0.979827,0.979557
5246,0.987409,0.986711,0.988627,1.000000,0.987773,0.986893,0.988829,0.989220,0.987582,0.986942,0.981786,0.981527
39373,0.987182,0.988790,0.987392,0.987773,1.000000,0.986844,0.988601,0.988647,0.987159,0.989687,0.978828,0.978637
10207,0.987452,0.988069,0.988808,0.986893,0.986844,1.000000,0.988811,0.988008,0.988063,0.987622,0.978305,0.978796
68163,0.988589,0.988719,0.988612,0.988829,0.988601,0.988811,1.000000,0.989800,0.988470,0.989247,0.979875,0.980678
80243,0.987683,0.987677,0.988219,0.989220,0.988647,0.988008,0.989800,1.000000,0.987920,0.988026,0.979685,0.980253
45052,0.988492,0.987590,0.987714,0.987582,0.987159,0.988063,0.988470,0.987920,1.000000,0.987935,0.979327,0.979208
99257,0.988120,0.989830,0.988696,0.986942,0.989687,0.987622,0.989247,0.988026,0.987935,1.000000,0.976154,0.976715


In [56]:
from sklearn.preprocessing import MinMaxScaler

In [57]:
scaler = MinMaxScaler()

In [58]:
cosine_similarities.min()

0.9761535325728232

In [59]:
min, max = (-1,1)
X_std = (cosine_similarities - cosine_similarities.min()) / (cosine_similarities.max() - cosine_similarities.min())
cosine_similarities_scaled = X_std * (max - min) + min

In [60]:
# np.fill_diagonal(cosine_similarities_scaled, np.nan)

In [61]:
cosine_similarities_df_scaled = pd.DataFrame(cosine_similarities_scaled,index = l2_norms_df.index, columns = l2_norms_df.index )

In [62]:
cosine_similarities_df_scaled

,34310,79613,3161,5246,39373,10207,68163,80243,45052,99257,health,random
34310,1.000000,0.005505,-0.052355,-0.055968,-0.075080,-0.052374,0.042993,-0.033012,0.034795,0.003646,-0.634609,-0.642015
79613,0.005505,1.000000,-0.040226,-0.114524,0.059779,-0.000631,0.053839,-0.033569,-0.040800,0.147066,-0.857657,-0.800759
3161,-0.052355,-0.040226,1.000000,0.046187,-0.057459,0.061361,0.044922,0.011913,-0.030413,0.051940,-0.691891,-0.714545
5246,-0.055968,-0.114524,0.046187,1.000000,-0.025479,-0.099305,0.063077,0.095919,-0.041480,-0.095171,-0.527618,-0.549303
39373,-0.075080,0.059779,-0.057459,-0.025479,1.000000,-0.103433,0.043981,0.047860,-0.076997,0.135073,-0.775732,-0.791752
10207,-0.052374,-0.000631,0.061361,-0.099305,-0.103433,1.000000,0.061606,-0.005746,-0.001186,-0.038156,-0.819544,-0.778346
68163,0.042993,0.053839,0.044922,0.063077,0.043981,0.061606,1.000000,0.144526,0.032984,0.098151,-0.687893,-0.620523
80243,-0.033012,-0.033569,0.011913,0.095919,0.047860,-0.005746,0.144526,1.000000,-0.013169,-0.004231,-0.703852,-0.656186
45052,0.034795,-0.040800,-0.030413,-0.041480,-0.076997,-0.001186,0.032984,-0.013169,1.000000,-0.011923,-0.733815,-0.743828
99257,0.003646,0.147066,0.051940,-0.095171,0.135073,-0.038156,0.098151,-0.004231,-0.011923,1.000000,-1.000000,-0.952885


In [63]:
cosine_similarities_df.to_csv('../results/cosine_similarities_multiple_runs.csv')
cosine_similarities_df_scaled.to_csv('../results/cosine_similarities_multiple_runs_scaled.csv')